In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision.utils import save_image
from tqdm import tqdm
import numpy as np
import time as time
import torchvision.transforms as T # Para manipular imagens para o FID
import torchvision.datasets as datasets # Para carregar o dataset real para o FID
import tempfile # Para criar diretórios temporários para imagens
import multiprocessing


In [ ]:
!pip install pytorch-fid

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from pytorch_fid.fid_score import calculate_fid_given_paths

In [ ]:
# Acesso ao diretório de dados do Kaggle, adaptado do seu notebook VAE
# Este trecho é específico para ambiente Kaggle. Fora dele, ajuste o caminho.
try:
    import kagglehub
    kushsheth_face_vae_path = kagglehub.dataset_download('kushsheth/face-vae')
    print('Data source import complete.')
except ImportError:
    print("kagglehub not found. Assuming local setup for data paths.")
    # Fallback for local development: Adjust these paths if running locally
    # For local setup, ensure you download the CelebA dataset and list_attr_celeba.csv
    # to these relative paths or specify absolute paths.
    kushsheth_face_vae_path = './data/face-vae' # Example local path
    if not os.path.exists(kushsheth_face_vae_path):
        print("Local data path not found. Please ensure CelebA dataset is available at expected location.")

100%|██████████| 1.33G/1.33G [00:15<00:00, 92.1MB/s]

Extracting files...


Data source import complete.


In [ ]:
# Hyperparameters (Ajustados para GANs e para o problema)
batch_size = 64 # Reduzido para testar, 128 também é comum
epochs = 25 # Aumentado para melhor convergência
latent_dim = 100 # Dimensão padrão para o vetor de ruído z
learning_rate_g = 0.0004 # Taxa de aprendizado para o Gerador
learning_rate_d = 0.00001 # Taxa de aprendizado para o Discriminador (TTUR: pode ser ajustada, ou mais passos D)
image_size = 128
channels = 3
num_attributes = 5 # Smiling, Male, Blond_Hair, Eyeglasses, Wearing_Hat
lambda_gp = 0.1 # Parâmetro para Gradient Penalty (WGAN-GP)
d_steps = 5 # Número de passos do Discriminador por passo do Gerador. Em WGAN, 5 é comum, mas vamos começar com 1 para estabilidade inicial.

# Definir os atributos-alvo
target_attrs = ['Smiling', 'Male', 'Blond_Hair', 'Eyeglasses', 'Wearing_Hat']

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, num_attributes, img_channels):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.num_attributes = num_attributes

        # Camada de embedding para os atributos
        # Mapeia o vetor de atributos para uma dimensão maior, que será concatenada ao ruído latente
        self.attr_embedding = nn.Sequential(
            nn.Linear(self.num_attributes, 128), # Ajuste a dimensão de saída conforme necessário
            nn.ReLU(True)
        )

        # Inicializa a camada de projeção para o início da rede convolucional
        # A entrada será latent_dim + dimensão do embedding dos atributos
        self.projection_dim = 512 * 4 * 4 # DCGAN starts with 512 filters at 4x4
        self.fc_projection = nn.Linear(self.latent_dim + 128, self.projection_dim)


        self.main = nn.Sequential(
            # Input: (latent_dim + num_attr_embedding) -> (512 * 4 * 4)
            # Reshape: (512, 4, 4)

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False), # 4x4 -> 8x8
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False), # 8x8 -> 16x16
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),  # 16x16 -> 32x32
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias=False),  # 32x32 -> 64x64
            nn.BatchNorm2d(32),
            nn.ReLU(True),

            nn.ConvTranspose2d(32, img_channels, 4, 2, 1, bias=False), # 64x64 -> 128x128
            nn.Tanh() # Saída da imagem normalizada para [-1, 1]
        )
        self.apply(weights_init)

    def forward(self, z, attrs):
        # Converte atributos para embedding
        attr_emb = self.attr_embedding(attrs)
        # Concatena ruído latente e embedding dos atributos
        z_conditioned = torch.cat([z, attr_emb], 1)

        # Projeta para o formato do input da primeira camada convolucional
        h = self.fc_projection(z_conditioned)
        h = F.relu(h) # ADICIONAR ESTA LINHA PARA MAIOR EXPRESSIVIDADE
        h = h.view(-1, 512, 4, 4) # Reshape para (N, C, H, W)

        return self.main(h)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, num_attributes, img_channels):
        super(Discriminator, self).__init__()
        self.num_attributes = num_attributes

        self.attr_embedding = nn.Sequential(
            nn.Linear(self.num_attributes, 128),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.feature_extractor = nn.Sequential(
            # Input: (img_channels, 128, 128)
            # >>>>> ADIÇÃO AQUI: Spectral Normalization <<<<<
            # Adicione SpectralNorm após cada camada Conv2d no Discriminator, exceto a primeira.
            # Isso ajuda a estabilizar o treinamento e pode mitigar o colapso de modo.
            nn.Conv2d(img_channels, 32, 4, 2, 1, bias=False), # 128x128 -> 64x64
            nn.LeakyReLU(0.2, inplace=True),

            nn.utils.spectral_norm(nn.Conv2d(32, 64, 4, 2, 1, bias=False)), # 64x64 -> 32x32
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),

            nn.utils.spectral_norm(nn.Conv2d(64, 128, 4, 2, 1, bias=False)), # 32x32 -> 16x16
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.utils.spectral_norm(nn.Conv2d(128, 256, 4, 2, 1, bias=False)), # 16x16 -> 8x8
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.utils.spectral_norm(nn.Conv2d(256, 512, 4, 2, 1, bias=False)), # 8x8 -> 4x4
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.fc_final = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 4 * 4 + 128, 1),
            # nn.Sigmoid()
        )
        self.apply(weights_init)

    def forward(self, img, attrs):
        attr_emb = self.attr_embedding(attrs)
        features = self.feature_extractor(img)
        features = features.view(features.size(0), -1)
        conditioned_features = torch.cat([features, attr_emb], 1)
        return self.fc_final(conditioned_features)

In [ ]:
# Inicializa modelos
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device being used: {device}')

netG = Generator(latent_dim, num_attributes, channels).to(device)
netD = Discriminator(num_attributes, channels).to(device)

# Otimizadores para WGAN-GP: Adam com beta1=0.0 para D
optimizerD = optim.Adam(netD.parameters(), lr=learning_rate_d, betas=(0.0, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=learning_rate_g, betas=(0.5, 0.999)) # Beta1 padrão para G

Device being used: cuda


In [1]:
# --- Dataset e DataLoader ---
class CelebAConditionalDataset(Dataset):
    def __init__(self, img_dir, attr_path, transform=None, target_attrs=None):
        self.img_dir = img_dir
        self.attributes_df = pd.read_csv(attr_path)
        self.transform = transform
        self.target_attrs = target_attrs

        # Certificar que todas as imagens referenciadas existem
        # self.image_files = [f for f in self.attributes_df.iloc[:, 0] if os.path.exists(os.path.join(self.img_dir, f))]
        # self.attributes_df = self.attributes_df[self.attributes_df.iloc[:, 0].isin(self.image_files)]


    def __len__(self):
        return len(self.attributes_df)

    def __getitem__(self, idx):
        img_name = self.attributes_df.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert('RGB') # Garante 3 canais

        if self.transform:
            image = self.transform(image)

        # Obter e converter os atributos para 0/1
        attrs = []
        if self.target_attrs:
            for attr in self.target_attrs:
                # O CelebA usa -1 para 'no' e 1 para 'yes'. Convertemos para 0 e 1.
                attrs.append(1 if self.attributes_df.iloc[idx][attr] == 1 else 0)

        # Converte a lista de atributos para um tensor float
        attrs_tensor = torch.tensor(attrs, dtype=torch.float32)

        return image, attrs_tensor

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normaliza para [-1, 1]
])

# Caminhos dos dados (adaptados para o diretório descompactado do KaggleHub ou local)
img_dir = os.path.join(kushsheth_face_vae_path, 'img_align_celeba', 'img_align_celeba')
attr_path = os.path.join(kushsheth_face_vae_path, 'list_attr_celeba.csv')

# Crie o dataset
dataset = CelebAConditionalDataset(
    img_dir=img_dir,
    attr_path=attr_path,
    transform=transform,
    target_attrs=target_attrs # Passa a lista de atributos
)

# Crie o DataLoader
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0, # Usar metade dos cores disponíveis
    pin_memory=True
)

# Verifica um lote de imagens e atributos
batch_images, batch_attrs = next(iter(dataloader))
print(f"Batch de Imagens shape: {batch_images.shape}") # Ex: [64, 3, 128, 128]
print(f"Batch de Atributos shape: {batch_attrs.shape}") # Ex: [64, 5]

# Plot the first 16 images with their attributes
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
axes = axes.ravel()
for i, ax in enumerate(axes):
    img = batch_images[i]
    img = img * 0.5 + 0.5 # Denormalize
    img = img.permute(1, 2, 0).cpu().numpy()
    ax.imshow(img)
    attr_labels = " ".join([f"{target_attrs[j]}: {'Y' if batch_attrs[i,j] == 1 else 'N'}" for j in range(num_attributes)])
    ax.set_title(attr_labels, fontsize=8)
    ax.axis('off')
plt.tight_layout()
plt.show()

# Cria diretórios para resultados
os.makedirs("results/cgan", exist_ok=True)
os.makedirs("models/cgan", exist_ok=True)

# Configuração do arquivo de log
log_file_path = "results/cgan/training_log.txt"
log_file = open(log_file_path, "w")
log_file.write("Iniciando o treinamento de CGAN com WGAN-GP\n")
log_file.write(f"Hyperparameters: Batch Size={batch_size}, Epochs={epochs}, Latent Dim={latent_dim}\n")
log_file.write(f"Learning Rate G={learning_rate_g}, Learning Rate D={learning_rate_d}\n")
log_file.write(f"Lambda GP={lambda_gp}, D Steps={d_steps}\n")
log_file.write("-" * 50 + "\n")

# --- Definição do Modelo CGAN (Baseado em DCGAN com Condicionamento) ---

# Função para inicializar pesos (melhora estabilidade em GANs)
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

NameError: name 'Dataset' is not defined

In [ ]:
# --- Função de Perda WGAN-GP ---
def calculate_gradient_penalty(discriminator, real_samples, fake_samples, attrs, lambda_gp, device):
    """Calcula o gradient penalty para WGAN-GP."""
    alpha = torch.rand(real_samples.size(0), 1, 1, 1, device=device)
    # Interpolação linear entre amostras reais e falsas
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    # Passa as interpolações e atributos pelo discriminador
    d_interpolates = discriminator(interpolates, attrs)

    # Calcula gradientes de d_interpolates em relação a interpolates
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=torch.ones_like(d_interpolates),
        create_graph=True,
        retain_graph=True,
        allow_unused=True
    )[0]

    gradients = gradients.view(gradients.size(0), -1)
    gradient_norm = gradients.norm(2, dim=1)
    gradient_penalty = ((gradient_norm - 1) ** 2).mean() * lambda_gp
    return gradient_penalty


In [45]:
# --- Configurações para FID ---
# Caminho para o dataset real (necessário para calcular o FID)
# Certifique-se de que este caminho esteja correto para o seu ambiente Colab/Kaggle
real_data_root = os.path.join(kushsheth_face_vae_path, 'img_align_celeba', 'img_align_celeba')

# Número de amostras para gerar e usar no cálculo do FID (deve ser um número razoável)
num_fid_samples = 10000  # Comece com um número menor para testes, pode aumentar para 10000+

# Transformação para as imagens que serão salvas para o cálculo do FID (normalmente [0,1] ou [0,255])
# O FID espera imagens RGB, normalizadas para [0, 1] ou [0, 255]
fid_image_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(), # Imagens em [0, 1]
])

# Função auxiliar para gerar e salvar imagens para o cálculo do FID
def generate_and_save_images_for_fid(generator, latent_dim, num_attributes, num_images, device, output_path, target_attrs_list):
    generator.eval()
    imgs_saved = 0
    while imgs_saved < num_images:
        batch_noise = torch.randn(min(num_images - imgs_saved, batch_size), latent_dim, device=device)
        # Gerar atributos aleatórios para a diversidade do FID
        batch_attrs = torch.randint(0, 2, (batch_noise.size(0), num_attributes), dtype=torch.float32).to(device)

        with torch.no_grad():
            fake_images = generator(batch_noise, batch_attrs).cpu()

        for j in range(fake_images.size(0)):
            img = fake_images[j]
            img = img * 0.5 + 0.5 # Denormaliza para [0, 1]
            save_image(img, os.path.join(output_path, f'generated_fid_{imgs_saved}.png'))
            imgs_saved += 1
            if imgs_saved >= num_images:
                break
    generator.train() # Voltar ao modo de treinamento
    return output_path

# Função auxiliar para preparar as imagens reais para o cálculo do FID
# Atenção: Esta função pode ser demorada na primeira execução se o dataset for grande
# e precisar ser copiado ou ter suas estatísticas calculadas.
def prepare_real_images_for_fid(data_root, num_images, output_path, transform):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        # Cria um dataset temporário para copiar as imagens reais
        # Para CelebA, carregar todas as imagens e copiar é ineficiente.
        # O pytorch-fid pode ler diretamente de um diretório.
        # O ideal é usar o DataLoader para pegar um subconjunto aleatório de imagens reais.

        # Solução mais simples e comum com pytorch-fid:
        # A biblioteca espera um diretório com imagens.
        # Podemos simplesmente usar o real_data_root.
        print(f"Usando imagens reais de: {data_root}")
    return data_root

# Lista para armazenar os valores de FID
fid_scores_per_epoch = []

In [ ]:
learning_rate_g = 0.0004
learning_rate_d = 0.00005
d_steps = 5
lambda_gp = 10.0

In [ ]:
# --- Treinamento ---
print("Iniciando o treinamento...")

fixed_noise = torch.randn(64, latent_dim, device=device)
fixed_attrs_base = torch.tensor([
    [1, 0, 0, 0, 0], # Smiling Female Dark NoGlasses NoHat
    [0, 1, 0, 0, 0], # NotSmiling Male Dark NoGlasses NoHat
    [1, 0, 1, 0, 0], # Smiling Female Blond NoGlasses NoHat
    [0, 1, 1, 0, 0], # NotSmiling Male Blond NoGlasses NoHat
    [1, 0, 0, 1, 0], # Smiling Female Dark Glasses NoHat
    [0, 1, 0, 1, 0], # NotSmiling Male Dark Glasses NoHat
    [1, 0, 0, 0, 1], # Smiling Female Dark NoGlasses Hat
    [0, 1, 0, 0, 1]  # NotSmiling Male Dark NoGlasses Hat
], dtype=torch.float32).to(device)

fixed_attrs = fixed_attrs_base.repeat(8, 1)

start_time = time.time()

g_losses_per_epoch = []
d_losses_per_epoch = []

best_g_loss = float('-inf') # Queremos a Loss G mais próxima de zero (menos negativa), então inicializamos com -infinito
best_epoch = 0

# >>> ADICIONADO PARA FID <<<
# Crie um diretório temporário para as imagens geradas para FID
temp_gen_fid_dir = tempfile.mkdtemp(prefix='fid_gen_')
# Use o diretório de dados reais diretamente, ou crie um subconjunto se preferir
real_fid_dir = real_data_root # Ou crie um subconjunto como em prepare_real_images_for_fid


for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()

    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc=f"Epoch {epoch}/{epochs}")

    current_d_loss_sum = 0
    current_g_loss_sum = 0

    # Adicionado um contador para os passos de G, para cálculo preciso da média
    g_steps_count = 0

    for i, (real_images, attrs) in pbar:
        log_file.write(f"Epoch {epoch}/{epochs}, Batch {i}: real_images min: {real_images.min().item():.4f}, max: {real_images.max().item():.4f}\n")
        real_images = real_images.to(device)
        attrs = attrs.to(device)
        batch_size = real_images.size(0)

        # Treinar o Discriminador
        netD.zero_grad()
        real_output = netD(real_images, attrs).view(-1)
        noise = torch.randn(batch_size, latent_dim, device=device)
        fake_images = netG(noise, attrs).detach()
        fake_output = netD(fake_images, attrs).view(-1)
        gp = calculate_gradient_penalty(netD, real_images.data, fake_images.data, attrs, lambda_gp, device)
        log_file.write(f"DEBUG: Iteration {i}, Gradient Penalty (GP): {gp.item():.4f}")
        d_loss = -torch.mean(real_output) + torch.mean(fake_output) + gp
        d_loss.backward()
        optimizerD.step()
        current_d_loss_sum += d_loss.item()

        # Treinar o Gerador (a cada d_steps do Discriminador)
        if i % d_steps == 0:
          netG.zero_grad()
          noise = torch.randn(batch_size, latent_dim, device=device)
          fake_images = netG(noise, attrs)

          # DEBUG CRÍTICO: Inspecionar a saída do Discriminador para imagens falsas
          g_output = netD(fake_images, attrs).view(-1)
          log_file.write(f"DEBUG: Iteration {i}, G_Output (Discriminator on Fakes) Sample: {g_output[0].item():.4f}, Min: {g_output.min().item():.4f}, Max: {g_output.max().item():.4f}, Mean: {g_output.mean().item():.4f}")

          g_loss = -torch.mean(g_output)
          log_file.write(f"DEBUG: Iteration {i}, G_Loss calculated: {g_loss.item():.4f}")

          g_loss.backward()
          # DEBUG CRÍTICO: Inspecionar gradientes do Gerador ANTES do optimizer.step()
          for name, param in netG.named_parameters():
              if param.grad is not None:
                  if (param.grad != 0).any(): # Verifica se há gradientes diferentes de zero
                      log_file.write(f"DEBUG: G Gradient for {name} (norm): {param.grad.norm().item():.6f}")
                      if param.grad.norm().item() > 1e6: # Gradiente explodindo
                          log_file.write(f"WARNING: Exploding gradient detected for {name} in Generator!")
                      elif param.grad.norm().item() < 1e-8: # Gradiente sumindo
                          log_file.write(f"WARNING: Vanishing gradient detected for {name} in Generator!")
              else:
                log_file.write(f"DEBUG: G Gradient for {name} is None (No gradient)") # Não deve acontecer se loss.backward() é chamado

          optimizerG.step()
          current_g_loss_sum += g_loss.item()
          g_steps_count += 1

        pbar.set_postfix({'Loss D': f'{d_loss.item():.4f}', 'Loss G': f'{g_loss.item():.4f}'})

    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time

    avg_d_loss = current_d_loss_sum / len(dataloader)
    # Usar g_steps_count para calcular a média da perda do Gerador
    avg_g_loss = current_g_loss_sum / g_steps_count if g_steps_count > 0 else 0

    g_losses_per_epoch.append(avg_g_loss)
    d_losses_per_epoch.append(avg_d_loss)

    total_time_elapsed = epoch_end_time - start_time
    avg_time_per_epoch = total_time_elapsed / epoch
    estimated_time_remaining = avg_time_per_epoch * (epochs - epoch)

    def format_time(seconds):
        m, s = divmod(seconds, 60)
        h, m = divmod(m, 60)
        return f"{int(h)}h {int(m)}m {int(s)}s"

    print(f"--- Epoch {epoch}/{epochs} Concluída ---")
    print(f"  Avg Loss D: {avg_d_loss:.4f} | Avg Loss G: {avg_g_loss:.4f}")
    print(f"  Duração da Época: {format_time(epoch_duration)}")
    print(f"  Tempo Total Decorrido: {format_time(total_time_elapsed)}")
    print(f"  Tempo Estimado Restante: {format_time(estimated_time_remaining)}")
    print(f"  Progresso Total: {(epoch/epochs)*100:.2f}%")

    with torch.no_grad():
        generated_samples = netG(fixed_noise, fixed_attrs).cpu()
        save_image(generated_samples, f'results/cgan/epoch_{epoch:03d}_generated_samples.png', nrow=8, normalize=True)

    # Calcular FID a cada X épocas (ex: a cada 5 épocas) para economizar tempo
    if epoch % 5 == 0: # Ajuste a frequência de cálculo do FID
        print("\nCalculando FID...")
        # Gerar e salvar imagens para o FID
        gen_path = generate_and_save_images_for_fid(netG, latent_dim, num_attributes, num_fid_samples, device, temp_gen_fid_dir, target_attrs)

        # Calcular FID
        # Certifique-se que o modelo Inception-v3 é baixado pela biblioteca (primeira vez)
        try:
            fid_value = calculate_fid_given_paths([real_fid_dir, gen_path],
                                                batch_size,
                                                device,
                                                dims=2048) # Dimensão das features do Inception
            fid_scores_per_epoch.append(fid_value)
            print(f"  FID Score na Época {epoch}: {fid_value:.2f}")
        except Exception as e:
            print(f"Erro ao calcular FID na Época {epoch}: {e}")
            fid_scores_per_epoch.append(float('nan')) # Adicionar NaN se houver erro

        # Limpar diretório temporário para a próxima avaliação FID
        import shutil
        shutil.rmtree(temp_gen_fid_dir)
        temp_gen_fid_dir = tempfile.mkdtemp(prefix='fid_gen_') # Recria o diretório

    # Salvar o melhor modelo (lógica baseada em G Loss)
    if avg_g_loss > best_g_loss:
        best_g_loss = avg_g_loss
        best_epoch = epoch
        torch.save(netG.state_dict(), 'models/cgan/best_generator.pth')
        torch.save(netD.state_dict(), 'models/cgan/best_discriminator.pth')
        print(f"  >>> Modelo salvo! Nova melhor G Loss: {best_g_loss:.4f} na Época {best_epoch} <<<")

    if epoch % 10 == 0:
        torch.save(netG.state_dict(), f'models/cgan/netG_epoch_{epoch:03d}.pth')
        torch.save(netD.state_dict(), f'models/cgan/netD_epoch_{epoch:03d}.pth')


print("\nTreinamento concluído.")

Iniciando o treinamento...


Epoch 1/25:  33%|███▎      | 1060/3166 [03:17<07:13,  4.86it/s, Loss D=-17.1699, Loss G=8.0363]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eb30078e0c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eb30078e0c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_wor

--- Epoch 1/25 Concluída ---
  Avg Loss D: -15.0282 | Avg Loss G: 19.2406
  Duração da Época: 0h 9m 49s
  Tempo Total Decorrido: 0h 9m 49s
  Tempo Estimado Restante: 3h 55m 40s
  Progresso Total: 4.00%
  >>> Modelo salvo! Nova melhor G Loss: 19.2406 na Época 1 <<<


Epoch 2/25:  34%|███▎      | 1061/3166 [03:15<06:28,  5.41it/s, Loss D=-14.4201, Loss G=28.7340]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eb30078e0c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eb30078e0c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_wo

--- Epoch 2/25 Concluída ---
  Avg Loss D: -12.1464 | Avg Loss G: 21.4268
  Duração da Época: 0h 9m 46s
  Tempo Total Decorrido: 0h 19m 35s
  Tempo Estimado Restante: 3h 45m 21s
  Progresso Total: 8.00%
  >>> Modelo salvo! Nova melhor G Loss: 21.4268 na Época 2 <<<


Epoch 3/25:  33%|███▎      | 1053/3166 [03:15<05:57,  5.91it/s, Loss D=-6.7507, Loss G=-1.7386]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eb30078e0c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eb30078e0c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_wor

--- Epoch 3/25 Concluída ---
  Avg Loss D: -12.1861 | Avg Loss G: 25.9041
  Duração da Época: 0h 9m 48s
  Tempo Total Decorrido: 0h 29m 25s
  Tempo Estimado Restante: 3h 35m 44s
  Progresso Total: 12.00%
  >>> Modelo salvo! Nova melhor G Loss: 25.9041 na Época 3 <<<


Epoch 4/25:  33%|███▎      | 1045/3166 [03:13<06:07,  5.77it/s, Loss D=-9.2342, Loss G=63.9261]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eb30078e0c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eb30078e0c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_wor

--- Epoch 4/25 Concluída ---
  Avg Loss D: -12.4176 | Avg Loss G: 30.8002
  Duração da Época: 0h 9m 45s
  Tempo Total Decorrido: 0h 39m 10s
  Tempo Estimado Restante: 3h 25m 40s
  Progresso Total: 16.00%
  >>> Modelo salvo! Nova melhor G Loss: 30.8002 na Época 4 <<<


Epoch 5/25:  33%|███▎      | 1045/3166 [03:14<08:05,  4.36it/s, Loss D=-7.9749, Loss G=34.6193]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eb30078e0c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7eb30078e0c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_wor

💹 *gráficos*

In [ ]:
total_training_duration = time.time() - start_time
print(f"Duração total do treinamento: {format_time(total_training_duration)}")

# Opcional: Plotar as perdas ao longo do tempo para visualização de progresso
plt.figure(figsize=(10, 5))
plt.plot(d_losses_per_epoch, label='Discriminator Loss')
plt.plot(g_losses_per_epoch, label='Generator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('GAN Training Losses Over Epochs')
plt.legend()
plt.grid(True)
plt.savefig('results/cgan/training_losses.png')
plt.show()


teste *dgfdfd*

In [ ]:
# prompt: ## generate image using model

import matplotlib.pyplot as plt
# Generate an image
def generate_image(generator, noise, attrs, device):
    """Generates an image using the trained generator."""
    generator.eval()  # Set generator to evaluation mode
    with torch.no_grad():
        noise = noise.to(device)
        attrs = attrs.to(device)
        generated_image = generator(noise, attrs)
        # Denormalize image
        generated_image = generated_image * 0.5 + 0.5
    generator.train() # Set generator back to training mode
    return generated_image.cpu() # Return to CPU

# Example usage: Generate one image with specific attributes
# Create a single noise vector
single_noise = torch.randn(1, latent_dim, device=device)

# Create a single attribute vector (e.g., Smiling, Male, Blond_Hair, No Eyeglasses, No Hat)
# Ensure the order matches target_attrs: ['Smiling', 'Male', 'Blond_Hair', 'Eyeglasses', 'Wearing_Hat']
desired_attrs = torch.tensor([[1, 1, 1, 0, 0]], dtype=torch.float32).to(device) # Example: Smiling Male Blond

# Generate the image
generated_single_image = generate_image(netG, single_noise, desired_attrs, device)

# Save or display the generated image
save_image(generated_single_image, 'results/cgan/single_generated_image.png')

# Display the image (optional)
plt.imshow(generated_single_image.squeeze().permute(1, 2, 0).numpy())
plt.title("Generated Image with Attributes: Smiling, Male, Blond Hair")
plt.axis('off')
plt.show()